# Functions

## Require

In [1]:
require 'json'
require "active_support/core_ext/hash"
require "active_support/core_ext/hash/indifferent_access"
require 'rubygems'
require 'zip'
require 'fileutils'
require 'faker'
puts

/Users/andreasbanholzer/.rvm/gems/ruby-2.7.1/gems/date-3.0.1/lib/date_core.bundle: warning: already initialized constant Date::MONTHNAMES
/Users/andreasbanholzer/.rvm/gems/ruby-2.7.1/gems/date-3.0.1/lib/date_core.bundle: warning: already initialized constant Date::ABBR_MONTHNAMES
/Users/andreasbanholzer/.rvm/gems/ruby-2.7.1/gems/date-3.0.1/lib/date_core.bundle: warning: already initialized constant Date::DAYNAMES
/Users/andreasbanholzer/.rvm/gems/ruby-2.7.1/gems/date-3.0.1/lib/date_core.bundle: warning: already initialized constant Date::ABBR_DAYNAMES
/Users/andreasbanholzer/.rvm/gems/ruby-2.7.1/gems/date-3.0.1/lib/date_core.bundle: warning: already initialized constant Date::ITALY
/Users/andreasbanholzer/.rvm/gems/ruby-2.7.1/gems/date-3.0.1/lib/date_core.bundle: warning: already initialized constant Date::ENGLAND
/Users/andreasbanholzer/.rvm/gems/ruby-2.7.1/gems/date-3.0.1/lib/date_core.bundle: warning: already initialized constant Date::JULIAN
/Users/andreasbanholzer/.rvm/gems/ruby-2

## header_create

In [2]:
def header_create( path, data )
    struct ={
        from: {
            name: '<script id="statosio_file">',
            index: nil
        },
        to: {
            name: "</script>",
            index: nil
        },
        file: {
            name: "var file = "
        }
    }
  
    file = File.read( path )
    puts struct[:from][:index] = file.index( struct[:from][:name] )
    struct[:from][:index] = struct[:from][:index] + struct[:from][:name].length
    tmp = file[ struct[:from][:index], file.length ]
    struct[:to][:index] = tmp.index( struct[:to][:name] )
    tmp = tmp[ 0, struct[:to][:index] ]
    struct[:file][:index] = tmp.index( struct[:file][:name] ) + struct[:file][:name].length
    str = ''
    str << file[ 0, struct[:from][:index] - struct[:from][:name].length  ]
    str << struct[:from][:name]
    str << "\n"
    str << struct[:file][:name]
    str << ''
    str << JSON.pretty_generate( data )
    str << ";\n"
    str << '</script>'
    
    return str
end

:header_create

# Execute

## Initialize

In [3]:
require './__generate/libs/helper'

path = './__generate/struct/hash.json'
hash = JSON.parse( File.read( path ) ).with_indifferent_access

require './__generate/libs/struct'
struct = struct_create( hash )

require './__generate/libs/blocks'
path = './__generate/struct/blocks.json'
blocks = blocks_create( path )

puts

## Options

In [6]:
require './__generate/libs/options'
diagrams = options_create( struct, blocks, :live, hash[:path][:root] )
puts 

## Benchmark

In [7]:
require './__generate/libs/benchmarks'
benchmarks_create( diagrams )

[256, 128, 64, 32, 16, 8, 4, 2, 1, 0]

## Anonymize Example Data

In [8]:
struct = {
    path: {
        performance: {
            from: nil,
            to: nil
        },
        header: {
            from: nil,
            to: nil
        }
    }
}

struct[:path][:performance][:from] = './__original/performance.json'
struct[:path][:performance][:to] = './data/performance.json'  

struct[:path][:header][:from] = './_includes/extra/head.html'
struct[:path][:header][:to] = './_includes/extra/head.html'

file = File.read( struct[:path][:performance][:from] )
data = JSON.parse( file )

names = 100.times.map { | d | Faker::TvShows::StarTrek.character }
[ [ 3, 'Spock' ], [ 12, 'Data' ] ].each { | d | names = names.insert( d[ 0 ], d[ 1 ] ) }
names = names.to_set.to_a

examples = []
data = data.each.with_index do | d, i |
    example = {
      name: nil,
      mobile: nil,
      desktop: nil
    }
    d["domain"] = names[ i ]  
    d["name"] = d.delete( "domain" )
  
    example.keys.each do | key |
        example[ key ] = d[ key.to_s ]
    end
    examples.push( example )
end

File.open( struct[:path][:performance][:to] , "w" ) { | f | f.write( JSON.pretty_generate( examples ) ) }

html = header_create( struct[:path][:header][:to], examples )
File.open( struct[:path][:header][:to], "w" ) { | f | f.write( html ) }
puts

1032

